# kostatiniyye 학습

## 라이브러리 로드

In [ ]:
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data import DataLoader

# 자체 라이브러리
from utils import model_controler
from utils import custom_logger
from models.kostantiniyye import Kostantiniyye

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## 데이터 로드

In [ ]:
touch = 'softness'

In [ ]:
train = torch.load(f'./dataset/{touch}_train.pth')
test = torch.load(f'./dataset/{touch}_test.pth')

train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=False)

In [ ]:
model = Kostantiniyye(device=device)
model.load_state_dict(torch.load(f'.checkpoints/pretrained_kostantiniyye.pth'), strict = False)

In [ ]:
model_controler.freeze_model(model, exception=
             ['attention', 'normalize'])
model.classifier.mobilenet.classifier[3].requires_grad = True
model.classifier.mobilenet.classifier[3].requires_grad 

In [ ]:
model.to(device)

In [ ]:
logger = custom_logger.get_logger(f'/content/drive/MyDrive/dataset/kostatiniyye_{touch}')

In [ ]:
# 손실 함수 및 최적화 함수 정의
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
num_epochs = 10
test_step = 1

In [ ]:
results = []

In [ ]:
logger.info(f'start new run lr:{lr}-------------')
result = run_train_test_2_input(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    best_test_loss = 999.,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    test_step=test_step,
    logger = logger,
    device = device,
    path = f'/content/best_kostantiniyye_{touch}.pth')
results.append(result)

In [ ]:
best_test_loss, train_losses, test_losses = [], [], []
for i in results:
    best_test_loss.append(i[0])
    train_losses += i[1]
    test_losses += i[2]

In [ ]:
with open(f'/content/drive/MyDrive/kostantiniyye_{touch}.pkl', 'wb') as f:
    pickle.dump(
        {
            'best_test_loss': best_test_loss,
            'train_losses': train_losses,
            'test_losses': test_losses
        }, f)